In [179]:
import numpy as np
import pandas as pd
#from google.colab import files
import matplotlib.pyplot as plt
import random
#need to install the pyhepmc module in order to import. python bindings to HEPMC
#!pip install pyhepmc
import pyhepmc as hep
import random as rd
#c_light  = 299.792458 mm/ns to get mm
c_light  = 299.792458

#Whether or not we move the signal event
shift=True

#Depending on the background type.
#SR Synchrotron radiation will be set too false since there will be many individual photons
#
one_background_particle=True



In [180]:
#container for signal events
sig_cont=[]
#container for background events
back_cont=[]
#container for combo events
combo_cont=[]

#opens up the signal file and extracts all individual events stores these events in sig_cont
#with hep.open("pythia8NCDIS_10x100_minQ2=1_beamEffects_xAngle=-0.025_hiDiv_1_000.hepmc") as f:
with hep.open("Test_DIS_event.hepmc") as f:
    for event in f:
        #print(event)
        sig_cont.append(event)
        pass

#opens up the signal file and extracts all individual events. stores these events in back_cont
#with hep.open("photon_event16_03_2023_02_05_02.hepmc") as f:
with hep.open("Test_Back_event.hepmc") as f:
    for event in f:
        #print(event)
        back_cont.append(event)
        pass





'''name='Test_DIS_event.hepmc'
with hep.WriterAsciiHepMC2(name) as f:
  for i in range(10):
    f.write_event(dis_event_cont[i])

name='Test_Back_event.hepmc'
with hep.WriterAsciiHepMC2(name) as f:
  for i in range(90,100):
    f.write_event(dis_event_cont[i])'''

"name='Test_DIS_event.hepmc'\nwith hep.WriterAsciiHepMC2(name) as f:\n  for i in range(10):\n    f.write_event(dis_event_cont[i])\n\nname='Test_Back_event.hepmc'\nwith hep.WriterAsciiHepMC2(name) as f:\n  for i in range(90,100):\n    f.write_event(dis_event_cont[i])"

In [181]:



for i in range(2):#len(sig_cont)):
    combo_cont.append(hep.GenEvent(hep.Units.GEV, hep.Units.MM))
    sig_particles=[]
    sig_vertices=[]
    sig_time_shift=c_light*(2000*rd.random())
    #sig_vertices.append(hep.GenVertex(sig_cont[i].event_pos()))
    
    #Stores the vertices of the event inside a vertex container. These vertices are in increasing order so we can index them with [abs(vertex_id)-1]
    for vertex in sig_cont[i].vertices:
        position=vertex.position
        #if we are shifting the event in time uses the random time generator and adds this time to the position four vector for the vertex
        if shift:
            position=position+hep.FourVector(x=0,y=0,z=0,t=sig_time_shift)
        v1=hep.GenVertex(position)
        sig_vertices.append(v1)
    print(len(sig_vertices))
    
    #copies the particles and attaches them to their corresponding vertices
    for particle in sig_cont[i].particles:
        momentum=particle.momentum
        status=particle.status
        pid=particle.pid
        
        # hep.GenParticle(momentum(px,py,pz,e), pdgid, status)
        p1 = hep.GenParticle(momentum,      pid,  status)
        p1.generated_mass = particle.generated_mass
        sig_particles.append(p1)
        #since the beam particles do not have a production vertex they cannot be attached to a production vertex
        if particle.production_vertex.id<0:
            production_vertex=particle.production_vertex.id
            sig_vertices[abs(production_vertex)-1].add_particle_out(p1)
            combo_cont[i].add_particle(p1)
        #Adds particles with an end vertex to their end vertices
        if (particle.end_vertex):     
            end_vertex=particle.end_vertex.id
            #print(end_vertex)
            sig_vertices[abs(end_vertex)-1].add_particle_in(p1)
    #Adds the vertices with the attached particles to the event        
    for vertex in sig_vertices:
        combo_cont[i].add_vertex(vertex)

    
    
    #Now we add in the background
    back_particles=[]
    back_vertices=[]
    
    #Standard use except for SR backgrounds. Will only shift the background event when this conditional is triggered.
    if one_background_particle:
        back_time_shift=c_light*(2000*rd.random())


    for vertex in back_cont[i].vertices:
        
        position=vertex.position
        #When the background event has many particles separated in time, like SR backgrounds, trigger this coniditional by changing the one_background_particle to False
        if one_background_particle==False:
            back_time_shift=c_light*(2000*rd.random())
        position=position+hep.FourVector(x=0,y=0,z=0,t=back_time_shift)
        v1=hep.GenVertex(position)
        back_vertices.append(v1)
    #print(len(back_vertices))
    
    #copies the particles and attaches them to their corresponding vertices
    for particle in back_cont[i].particles:
        momentum=particle.momentum
        status=particle.status
        pid=particle.pid
        
        # hep.GenParticle(momentum(px,py,pz,e), pdgid, status)
        p1 = hep.GenParticle(momentum,      pid,  status)
        p1.generated_mass = particle.generated_mass
        back_particles.append(p1)
        #since the beam particles do not have a production vertex they cannot be attached to a production vertex
        if particle.production_vertex.id<0:
            production_vertex=particle.production_vertex.id
            back_vertices[abs(production_vertex)-1].add_particle_out(p1)
            combo_cont[i].add_particle(p1)
        #Adds particles with an end vertex to their end vertices
        if (particle.end_vertex):     
            end_vertex=particle.end_vertex.id
            #print(end_vertex)
            back_vertices[abs(end_vertex)-1].add_particle_in(p1)
    #Adds the vertices with the attached particles to the event        
    for vertex in back_vertices:
        combo_cont[i].add_vertex(vertex)
'''print(sig_cont[0])
print(back_cont[0])
print(combo_cont[0])'''







8
8
________________________________________________________________________
GenEvent: #0
 Momentum units: GEV Position units: MM
 Entries in this event: 8 vertices, 14 particles, 0 weights.
 Position offset: 0, 0, 0, 0
                                    GenParticle Legend
         ID    PDG ID   ( px,       py,       pz,     E )   Stat ProdVtx
________________________________________________________________________
Vtx:     -1 stat:   0 (X,cT): -0.23 0.001 39 -32
 I:       1     2212 -2.50e+00,+3.87e-02,+1.00e+02,+1.00e+02   4     0
 O:       2       -2 -3.05e-01,+4.41e-01,+9.80e+00,+9.82e+00  61    -1
          3     2212 -2.38e+00,-3.88e-01,+6.84e+01,+6.85e+01   1    -1
          4        2 -5.84e-01,-3.06e-01,+2.28e+01,+2.28e+01  63    -1
Vtx:     -2 stat:   0 (X,cT): -0.23 0.001 39 -32
 I:       5       11 +3.65e-03,+1.96e-04,-1.00e+01,+1.00e+01   4     0
 O:       6       11 +3.65e-03,+1.96e-04,-1.00e+01,+1.00e+01  21    -2
Vtx:     -3 stat:   0 (X,cT): -0.23 0.001 39 -32
 I:   

In [182]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
print("date and time =", dt_string)	
#name='combo_event{}.hepmc'.format(dt_string)

name='Dis_1000_event{}.hepmc'.format(dt_string)


with hep.WriterAsciiHepMC2(name) as f:
  for i in combo_cont:
    f.write_event(i)

now = 2023-03-27 01:32:57.157333
date and time = 27_03_2023_01_32_57


C:\Users\cello\AppData\Local\Temp\ipykernel_21648\3086032701.py:16: VisibleDeprecationWarning: importing WriterAsciiHepMC2 from pyhepmc is deprecated, please import from pyhepmc.io
  with hep.WriterAsciiHepMC2(name) as f:
